# Fedbiomed Researcher base example with OPACUS

Use for developing (autoreloads changes made across packages)

In this notebook we show how `opacus` (https://opacus.ai/) can be used in Fed-BioMed. Opacus is a library which allows to train PyTorch models with differential privacy. We will train the basic MNIST example using two nodes.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)

2022-01-18 13:16:12,975 fedbiomed INFO - Component environment:
2022-01-18 13:16:12,976 fedbiomed INFO - - type = ComponentType.RESEARCHER
2022-01-18 13:16:14,536 fedbiomed INFO - Messaging researcher_ff08ab68-4655-4552-b2f2-53ade0922f4f successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x10cd0f970>
2022-01-18 13:16:14,575 fedbiomed INFO - Listing available datasets in all nodes... 
2022-01-18 13:16:14,589 fedbiomed INFO - log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / DEBUG - Message received: {'researcher_id': 'researcher_ff08ab68-4655-4552-b2f2-53ade0922f4f', 'command': 'list'}
2022-01-18 13:16:14,600 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / DEBUG - Message received: {'researcher_id': 'researcher_ff08ab68-4655-4552-b2f2-53ade0922f4f', 'command': 'list'}
2022-01-18 13:16:24,625 fedbiomed INFO - 
 Node: node_c23cd342-690d-4908-bee6-1add6f8c2907 | Number of Datasets: 1 
+--------+-------------+---

{'node_c23cd342-690d-4908-bee6-1add6f8c2907': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': 'MNIST database',
   'shape': [60000, 1, 28, 28]}],
 'node_9afc3a65-86d6-43da-9aaa-a7fedf79887d': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': 'MNIST database',
   'shape': [60000, 1, 28, 28]}]}

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default), and write MNIST to add MNIST to the node through `torchvision.datasets.CIFAR10`
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Define an experiment model and parameters

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [3]:
from fedbiomed.researcher.environ import environ
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+'/')
model_file = tmp_dir_model.name + '/class_export_mnist.py'

In the cell below, we are going to define the model using opacus for differential privacy. For this example, we are going to use the function `make_private` from `opacus.privacy_engine`. Two hyperparameters should be defined:
* `noise_multiplier`: The ratio of the standard deviation of the Gaussian noise to the L2-sensitivity of the function to which the noise is added (How much noise to add)
* `max_grad_norm`: The maximum norm of the per-sample gradients. Any gradient with norm higher than this will be clipped to this value.

It is worth noting that in order to use the opacus `PrivacyEngine` class we need to properly define as training plan attributes a `model`, a `dataloader` and an `optimizer`.

In [4]:
%%writefile "$model_file"

import torch
import torch.nn as nn
import torch.nn.functional as F
from fedbiomed.common.torchnn import TorchTrainingPlan
from fedbiomed.common.logger import logger
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from opacus import PrivacyEngine 
from typing import Union, List
from tqdm import tqdm

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args):
        super(MyTrainingPlan, self).__init__()
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms",
                "import torch.nn.functional as F",
                "from torch.utils.data import DataLoader",
                "from fedbiomed.common.logger import logger",
                "from typing import Union, List",
                "from tqdm import tqdm",
                "from opacus import PrivacyEngine",]
        self.add_dependency(deps)
        
        self.model = nn.Sequential(nn.Conv2d(1, 32, 3, 1),
                                  nn.ReLU(),
                                  nn.Conv2d(32, 64, 3, 1),
                                  nn.ReLU(),
                                  nn.MaxPool2d(2),
                                  nn.Dropout(0.25),
                                  nn.Flatten(),
                                  nn.Linear(9216, 128),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(128, 10),
                                  nn.LogSoftmax(dim=1))
        
        self.noise_multiplier = model_args['noise_multiplier']
        self.max_grad_norm = model_args['max_grad_norm']

    def forward(self, x):
        return self.model(x)

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss
    
    def training_routine(self,
                         epochs: int = 2,
                         log_interval: int = 10,
                         lr: Union[int, float] = 1e-3,
                         batch_size: int = 48,
                         batch_maxnum: int = 0,
                         dry_run: bool = False,
                         monitor=None):
        
        #if self.optimizer is None:
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

        # use_cuda = torch.cuda.is_available()
        # device = torch.device("cuda" if use_cuda else "cpu")
        self.device = "cpu"
        
        training_data = self.training_data(batch_size=batch_size)
        
        # enter PrivacyEngine
        privacy_engine = PrivacyEngine()
        self.model, self.optimizer, training_data = privacy_engine.make_private(module=self.model,
                                                                    optimizer=self.optimizer,
                                                                    data_loader=training_data,
                                                                    noise_multiplier=self.noise_multiplier,
                                                                    max_grad_norm=self.max_grad_norm,
                                                                    )

        for epoch in range(1, epochs + 1):
            # (below) sampling data (with `training_data` method defined on
            # researcher's notebook)
            for batch_idx, (data, target) in enumerate(tqdm(training_data)):
                #self.model.train()  # model training
                data, target = data.to(self.device), target.to(self.device)
                self.optimizer.zero_grad()
                # (below) calling method `training_step` defined on
                # researcher's notebook
                res = self.training_step(data, target)
                res.backward()
                self.optimizer.step()

                # do not take into account more than batch_maxnum
                # batches from the dataset
                if (batch_maxnum > 0) and (batch_idx >= batch_maxnum):
                    #print('Reached {} batches for this epoch, ignore remaining data'.format(batch_maxnum))
                    logger.debug('Reached {} batches for this epoch, ignore remaining data'.format(batch_maxnum))
                    break

                if batch_idx % log_interval == 0:
                    logger.info('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch,
                        batch_idx * len(data),
                        len(training_data.dataset),
                        100 * batch_idx / len(training_data),
                        res.item()))

                    # Send scalar values via general/feedback topic
                    if monitor is not None:
                        monitor.add_scalar('Loss', res.item(), batch_idx, epoch)

                    if dry_run:
                        return

    def save(self, filename, params: dict = None) -> None:
        if params is not None:

            # params keys are changed by the privacy engine (as _module.param_key): should be re-named
            params_keys = list(params.keys())
            for key in params_keys:
                if '_module' in key:
                    newkey = key.replace('_module.', '')
                    params[newkey] = params.pop(key)
                    
            return(torch.save(params, filename))
        else:
            return torch.save(self.state_dict(), filename)

Writing /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/tmp_oxfl314/class_export_mnist.py


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side. For instance, the privacy parameters should be passed here.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [5]:
model_args = {'noise_multiplier':1., 'max_grad_norm':1.0}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 3, 
    'dry_run': False,  
    'batch_maxnum': 250 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `rounds` rounds, applying the `node_selection_strategy` between the rounds

In [6]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 3

exp = Experiment(tags=tags,
                 #nodes=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-01-18 13:16:33,727 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-01-18 13:16:33,745 fedbiomed INFO - log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / DEBUG - Message received: {'researcher_id': 'researcher_ff08ab68-4655-4552-b2f2-53ade0922f4f', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-01-18 13:16:33,754 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / DEBUG - Message received: {'researcher_id': 'researcher_ff08ab68-4655-4552-b2f2-53ade0922f4f', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-01-18 13:16:43,735 fedbiomed INFO - Node selected for training -> node_9afc3a65-86d6-43da-9aaa-a7fedf79887d
2022-01-18 13:16:43,736 fedbiomed INFO - Node selected for training -> node_c23cd342-690d-4908-bee6-1add6f8c2907
2022-01-18 13:16:43,739 fedbiomed INFO - Checking data quality of federated datasets...
2022-01-18 13:16:44,364 fedbiomed DEBUG - torchnn saved model filename: /Users/balelli/o

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the nodes

In [7]:
exp.run()

2022-01-18 13:16:49,079 fedbiomed INFO - Sampled nodes in round 0 ['node_9afc3a65-86d6-43da-9aaa-a7fedf79887d', 'node_c23cd342-690d-4908-bee6-1add6f8c2907']
01/18/2022 13:16:49:INFO:Sampled nodes in round 0 ['node_9afc3a65-86d6-43da-9aaa-a7fedf79887d', 'node_c23cd342-690d-4908-bee6-1add6f8c2907']
2022-01-18 13:16:49,081 fedbiomed INFO - Send message to node node_9afc3a65-86d6-43da-9aaa-a7fedf79887d - {'researcher_id': 'researcher_ff08ab68-4655-4552-b2f2-53ade0922f4f', 'job_id': '2160dcbf-59be-4190-aabf-e29aad51051a', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {'noise_multiplier': 1.0, 'max_grad_norm': 1.0}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/18/my_model_ecdd2f8d-8031-47ec-8d54-6e4d9ace7026.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/18/aggregated_params_init_56ccdbb9-c78c-43c3-bc62-c8aa40f21648.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'n

2022-01-18 13:16:49,206 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_ff08ab68-4655-4552-b2f2-53ade0922f4f', 'job_id': '2160dcbf-59be-4190-aabf-e29aad51051a', 'params_url': 'http://localhost:8844/media/uploads/2022/01/18/aggregated_params_init_56ccdbb9-c78c-43c3-bc62-c8aa40f21648.pt', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'training_data': {'node_c23cd342-690d-4908-bee6-1add6f8c2907': ['dataset_3fc920be-14db-4941-b8ae-90f436605b11']}, 'model_args': {'noise_multiplier': 1.0, 'max_grad_norm': 1.0}, 'model_url': 'http://localhost:8844/media/uploads/2022/01/18/my_model_ecdd2f8d-8031-47ec-8d54-6e4d9ace7026.py', 'model_class': 'MyTrainingPlan', 'command': 'train'}
01/18/2022 13:16:49:INFO:log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_ff08ab68-4655-4552-b2f2-53ade0922f4f', 'job_id': '21

2022-01-18 13:18:02,936 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 1 [4500/60000 (7%)]	Loss: 2.052957
01/18/2022 13:18:02:INFO:log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 1 [4500/60000 (7%)]	Loss: 2.052957
2022-01-18 13:18:03,221 fedbiomed INFO - log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 1 [4860/60000 (7%)]	Loss: 2.103401
01/18/2022 13:18:03:INFO:log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 1 [4860/60000 (7%)]	Loss: 2.103401
2022-01-18 13:18:10,247 fedbiomed INFO - log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 1 [4400/60000 (8%)]	Loss: 2.071974
01/18/2022 13:18:10:INFO:log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 1 [4400/60000 (8%)]	Loss: 2.071974
2022-01-18 13:18:10,576 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 1 [5300/60000 (8%)]	Loss: 2.06824

2022-01-18 13:19:43,612 fedbiomed INFO - log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 1 [8510/60000 (18%)]	Loss: 1.688652
01/18/2022 13:19:43:INFO:log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 1 [8510/60000 (18%)]	Loss: 1.688652
2022-01-18 13:19:48,502 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 1 [9120/60000 (19%)]	Loss: 1.874794
01/18/2022 13:19:48:INFO:log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 1 [9120/60000 (19%)]	Loss: 1.874794
2022-01-18 13:19:50,060 fedbiomed INFO - log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 1 [12000/60000 (19%)]	Loss: 1.573788
01/18/2022 13:19:50:INFO:log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 1 [12000/60000 (19%)]	Loss: 1.573788
2022-01-18 13:19:55,093 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / DEBUG - Reached 250 batches for this epoch, i

2022-01-18 13:21:12,667 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 2 [4920/60000 (10%)]	Loss: 1.342297
01/18/2022 13:21:12:INFO:log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 2 [4920/60000 (10%)]	Loss: 1.342297
2022-01-18 13:21:16,682 fedbiomed INFO - log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 2 [5880/60000 (10%)]	Loss: 1.337237
01/18/2022 13:21:16:INFO:log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 2 [5880/60000 (10%)]	Loss: 1.337237
2022-01-18 13:21:19,411 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 2 [8060/60000 (10%)]	Loss: 1.405492
01/18/2022 13:21:19:INFO:log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 2 [8060/60000 (10%)]	Loss: 1.405492
2022-01-18 13:21:23,146 fedbiomed INFO - log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 2 [6370/60000 (10%)]	Loss: 

...

01/18/2022 13:39:15:INFO:log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 1 [13200/60000 (18%)]	Loss: 2.071954
2022-01-18 13:39:18,127 fedbiomed INFO - log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 1 [9680/60000 (18%)]	Loss: 0.853043
01/18/2022 13:39:18:INFO:log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 1 [9680/60000 (18%)]	Loss: 0.853043
2022-01-18 13:39:21,967 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 1 [11270/60000 (18%)]	Loss: 1.301942
01/18/2022 13:39:21:INFO:log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 1 [11270/60000 (18%)]	Loss: 1.301942
2022-01-18 13:39:24,567 fedbiomed INFO - log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 1 [14490/60000 (18%)]	Loss: 1.331417
01/18/2022 13:39:24:INFO:log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 1 [14490/60000 (18%)]	Loss: 1.331417
20

01/18/2022 13:40:40:INFO:log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 2 [4700/60000 (8%)]	Loss: 1.422997
2022-01-18 13:40:42,961 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 2 [5390/60000 (9%)]	Loss: 0.939601
01/18/2022 13:40:42:INFO:log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 2 [5390/60000 (9%)]	Loss: 0.939601
2022-01-18 13:40:46,875 fedbiomed INFO - log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 2 [3960/60000 (9%)]	Loss: 0.787530
01/18/2022 13:40:46:INFO:log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 2 [3960/60000 (9%)]	Loss: 0.787530
2022-01-18 13:40:49,550 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 2 [6600/60000 (10%)]	Loss: 1.243998
01/18/2022 13:40:49:INFO:log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 2 [6600/60000 (10%)]	Loss: 1.243998
2022-01-18 1

01/18/2022 13:42:12:INFO:log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / DEBUG - Reached 250 batches for this epoch, ignore remaining data
2022-01-18 13:42:12,758 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 3 [0/60000 (0%)]	Loss: 1.571053
01/18/2022 13:42:12:INFO:log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 3 [0/60000 (0%)]	Loss: 1.571053
2022-01-18 13:42:15,834 fedbiomed INFO - log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / DEBUG - Reached 250 batches for this epoch, ignore remaining data
01/18/2022 13:42:15:INFO:log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / DEBUG - Reached 250 batches for this epoch, ignore remaining data
2022-01-18 13:42:16,490 fedbiomed INFO - log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 3 [0/60000 (0%)]	Loss: 0.995400
01/18/2022 13:42:16:INFO:log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 3 [0/60000 (0%)]	Loss: 0.

01/18/2022 13:43:42:INFO:log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 3 [5980/60000 (10%)]	Loss: 2.601202
2022-01-18 13:43:46,086 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 3 [7280/60000 (11%)]	Loss: 0.948497
01/18/2022 13:43:46:INFO:log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 3 [7280/60000 (11%)]	Loss: 0.948497
2022-01-18 13:43:49,539 fedbiomed INFO - log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 3 [6720/60000 (11%)]	Loss: 0.940858
01/18/2022 13:43:49:INFO:log from: node_9afc3a65-86d6-43da-9aaa-a7fedf79887d / INFO - Train Epoch: 3 [6720/60000 (11%)]	Loss: 0.940858
2022-01-18 13:43:52,595 fedbiomed INFO - log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 3 [6600/60000 (12%)]	Loss: 1.354029
01/18/2022 13:43:52:INFO:log from: node_c23cd342-690d-4908-bee6-1add6f8c2907 / INFO - Train Epoch: 3 [6600/60000 (12%)]	Loss: 1.354029
2022-01

01/18/2022 13:45:14:INFO:Downloading model params after training on node_9afc3a65-86d6-43da-9aaa-a7fedf79887d - from http://localhost:8844/media/uploads/2022/01/18/node_params_bc182a4e-490a-436f-b75d-1cbbe48befef.pt
2022-01-18 13:45:15,094 fedbiomed INFO - Nodes that successfully reply in round 2 ['node_c23cd342-690d-4908-bee6-1add6f8c2907', 'node_9afc3a65-86d6-43da-9aaa-a7fedf79887d']
01/18/2022 13:45:15:INFO:Nodes that successfully reply in round 2 ['node_c23cd342-690d-4908-bee6-1add6f8c2907', 'node_9afc3a65-86d6-43da-9aaa-a7fedf79887d']
2022-01-18 13:45:15,871 fedbiomed INFO - Saved aggregated params for round 2 in /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0000/aggregated_params_4398590d-1545-4040-ab7e-b7841b1731fd.pt
01/18/2022 13:45:15:INFO:Saved aggregated params for round 2 in /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0000/aggregated_params_4398590d-1545-4040-ab7e-b7841b1731fd.pt


Local training results for each round and each node are available in `exp.training_replies` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [8]:
print("\nList the training rounds : ", exp.training_replies.keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies[rounds - 1].data
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies[rounds - 1].dataframe


List the training rounds :  dict_keys([0, 1, 2])

List the nodes for the last training round and their timings : 
	- node_c23cd342-690d-4908-bee6-1add6f8c2907 :    
		rtime_training=496.12 seconds    
		ptime_training=673.49 seconds    
		rtime_total=510.51 seconds
	- node_9afc3a65-86d6-43da-9aaa-a7fedf79887d :    
		rtime_training=499.33 seconds    
		ptime_training=676.23 seconds    
		rtime_total=510.80 seconds




,success,msg,dataset_id,node_id,params_path,params,timing
0,True,,dataset_3fc920be-14db-4941-b8ae-90f436605b11,node_c23cd342-690d-4908-bee6-1add6f8c2907,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'model._module.0.weight': [[tensor([[-0.0592,...","{'rtime_training': 496.12184728700004, 'ptime_..."
1,True,,dataset_77b58df6-6d02-45a4-9b70-81d44e34d6df,node_9afc3a65-86d6-43da-9aaa-a7fedf79887d,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'model._module.0.weight': [[tensor([[-0.0198,...","{'rtime_training': 499.33108151500005, 'ptime_..."


Federated parameters for each round are available in `exp.aggregated_params` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [9]:
print("\nList the training rounds : ", exp.aggregated_params.keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params[rounds - 1]['params'].keys())



List the training rounds :  dict_keys([0, 1, 2])

Access the federated params for the last training round :
	- params_path:  /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0000/aggregated_params_4398590d-1545-4040-ab7e-b7841b1731fd.pt
	- parameter data:  odict_keys(['model.0.weight', 'model.0.bias', 'model.2.weight', 'model.2.bias', 'model.7.weight', 'model.7.bias', 'model.10.weight', 'model.10.bias'])


# Testing

We define a little testing routine to extract the accuracy metrics on the testing dataset

In [10]:
import torch
import torch.nn.functional as F


def testing_Accuracy(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    device = 'cpu'

    correct = 0
    
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

        pred = output.argmax(dim=1, keepdim=True)

    test_loss /= len(data_loader.dataset)
    accuracy = 100* correct/len(data_loader.dataset)

    return(test_loss, accuracy)

In [11]:
from torchvision import datasets, transforms
import os

local_mnist = os.path.join(environ['TMP_DIR'], 'local_mnist')

transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])

test_set = datasets.MNIST(root = local_mnist, download = True, train = False, transform = transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True)

fed_model = exp.model_instance
fed_model.load_state_dict(exp.aggregated_params[rounds - 1]['params'])

acc_federated = testing_Accuracy(fed_model, test_loader)

print('\nAccuracy federated training:  {:.4f}'.format(acc_federated[1]))

print('\nError federated training:  {:.4f}'.format(acc_federated[0]))

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw/train-images-idx3-ubyte.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw

Processing...


/Users/balelli/miniconda3/envs/fedbiomed-researcher/lib/python3.9/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!

Accuracy federated training:  86.4300

Error federated training:  0.5694
